In [1]:
from selenium import webdriver
import requests
from bs4 import BeautifulSoup as bs
from selenium.webdriver import ActionChains as ac
import pandas as pd
import time
from selenium.webdriver.support.select import Select

In [2]:
driver = webdriver.Chrome('./chromedriver')
driver.implicitly_wait(5)
driver.get('https://www.agoda.com/ko-kr/search?city=14690&locale=ko-kr&ckuid=85bd8e58-3e28-4b77-bbc9-220d4005678d&prid=0&currency=KRW&correlationId=53d841b7-667a-46d2-9254-9f50dbb613d6&pageTypeId=103&realLanguageId=9&languageId=9&origin=KR&cid=1829971&tag=747e9723-83df-6f99-853a-8505c70753f7&userId=85bd8e58-3e28-4b77-bbc9-220d4005678d&whitelabelid=1&loginLvl=0&storefrontId=3&currencyId=26&currencyCode=KRW&htmlLanguage=ko-kr&cultureInfoName=ko-kr&machineName=hk-crweb-2011&trafficGroupId=5&sessionId=zhcjc1dv5s1tlmr15vkzg1xv&trafficSubGroupId=122&aid=82361&useFullPageLogin=true&cttp=4&isRealUser=true&mode=production&checkIn=2021-07-08&checkOut=2021-07-09&rooms=1&adults=2&children=0&priceCur=KRW&los=1&textToSearch=%EC%84%9C%EC%9A%B8&productType=-1&travellerType=1&familyMode=off')
html = driver.page_source
soup = bs(html, 'html.parser')

In [6]:
driver.find_element_by_css_selector('h3.PropertyCard__HotelName').click()


In [14]:
# 리뷰 필터 선택
button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[3]/div[1]')
                                      
ac(driver).move_to_element(button).click().perform
time.sleep(2)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@id="reviewFilterSection"]/div[3]/div[1]"}
  (Session info: chrome=91.0.4472.124)


In [ ]:
ac(driver).move_to_element(scroll_point).perform()


In [ ]:

html = driver.page_source
soup = bs(html, 'html.parser')

button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[3]/div[1]')
                                      
ac(driver).move_to_element(button).click().perform
time.sleep(2)
button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[3]/div[1]')
ac(driver).move_to_element(button).click().perform
time.sleep(2)
button = driver.find_element_by_xpath('//*[@id="reviewFilterSection"]/div[1]/div[3]')
ac(driver).move_to_element(button).click().perform()
time.sleep(2)

In [4]:
#한국어클릭
button = driver.find_element_by_class_name('//*[@id="revi"]/div[1]/div[3]/div')
ac(driver).move_to_element(button).click().perform()
time.sleep(2)

driver.implicitly_wait(5)

In [5]:
# 다시 파싱
html = driver.page_source
soup = bs(html, 'html.parser')

In [6]:
#총 리뷰 개수 찾기 : for문을 몇 번 돌릴지 구하기위함 (버튼을 몇 번 눌러야 할지) 
html = driver.page_source
soup = bs(html, 'html.parser')
review_sum0 =  soup.select('.Review__FilterContainer__Dropbox > span') #이게 아니었네용
review_sum0 = review_sum0[0].text.split("(")[1][:-1]

#실제 이용후기 개수
review_sum = soup.select('span.Review__SummaryContainer--left.Review__SummaryContainer__Text')[0].text
review_sum = review_sum.split(" ")[3][:-1]


pages = int(review_sum)//20 + 1

In [7]:
# 데이터프레임으로 만들 딕셔너리 정의
reviews = {'text':[]}

In [8]:
#page 넘어가기
for k in range(1,pages+1):
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    # 리뷰 영역까지 스크롤
    scroll_point = driver.find_element_by_id('review-0')
    ac(driver).move_to_element(scroll_point).perform()
    
    # 다시 파싱
    html = driver.page_source
    soup = bs(html, 'html.parser')
    
    for k in range(0,9):
        date_ = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-statusBar > div > div > span')
     
        year = date_[0].text.split(" ")[1][:-1]
        if int(year) >= 2018:
            text = soup.select(f'#review-{k} > div.Review-comment-right > div.Review-comment-bubble > div.Review-comment-body > p.Review-comment-bodyText')
            
            reviews['text'].append(text[0].text)
        else:
            break
            

    #버튼누르기
    button = driver.find_element_by_xpath('//*[@id="reviewSection"]/div[4]/div/span[3]')
    ac(driver).move_to_element(button).click().perform()
    time.sleep(2)

In [9]:
#호텔명
hotel = soup.select('.HeaderCerebrum__Name')
hotel

[<h1 class="HeaderCerebrum__Name" data-selenium="hotel-header-name">글래드라이브 호텔 (Glad Live Gangnam)</h1>]

In [10]:
#호텔명
reviews['title'] = hotel[0].text

In [11]:
reviews

{'text': ['ㅇ',
  '글래드 계열 호텔 침구은 언제나 편안하고, 룸 청결 상태는 항상 좋습니다~',
  '오랜전부터 출장 때문에 자주 투숙하고 있습니다. 침대도 편안하고 모든게 좋은데 가격은 동일함에도 불구하고 점점 방문할수록 어매니티가 하나씩 사라져서 방문할 때마다 당황스럽네요^^',
  '지하철 역이랑 가까워서 좋아요',
  '넓은 침대와 긴 쇼파 2개로 5명까지 우리는 편하게 잤다.',
  '방이 깔끔하고 분위기도 좋고 주변에 맛집도 많아서 좋았습니다. 특히 블루투스스피커도 있어서 더욱 호캉스기분 났습니다~',
  '역에서 가깝고 좋아요',
  '위치도 좋고 숙소 상태도 깨끗해서 또 오고 싶네요\n고층 요청했더니 19층 주셨어요 감동~\n다음에는 파티룸을 이용할까해요 ',
  '강남에서 이 가격에 \n작은 파티할 만한 크기와 무드 \n분리된 거실을 가진 호텔은 여기뿐임.\n부대시설은 이용할 만한 것이 없지만\n주변 맛집 많고, 배달 안 되는 음식이 없다.',
  '너무좋아요! ',
  '서울에서의 시험을 위해 묵었던 호텔이였던만큼 밤에 시끄럽지는 않을까 라는 우려가 있었습니다만 우려와는 달리 너무 조용하고 좋았습니다 방도 깔끔하고 직원분들의대응도 좋았습니다 덕분에 컨디션조절도 잘되어 시험도 잘봤습니다:) 감사합니다~',
  '저렴하게 잘 이용하다 갑니다',
  '트윈룸예약! 룸컨디션좋고 침구좋고 디자인좋고 특히 샤워가운 재질이 너무마음에듬! 밤에 밖에 길거리나 호텔 내에 소음? 다른방 사람소리? 좀 시끄럽다 생각했고 잠자는데는 문제없을정도였음 체크아웃시간맞춰 체크아웃했는데 주차타워점검중이라고 20분정도 기다림 레스토랑 런치예약땜에 시간맞춰나왔는데 생각지도못한 딜레이에.. 레스토랑에 양해구하고 예약변경함 굳이 체크아웃시간에 점검했어야하는지의문 호텔투숙객인데 일찍가서 주차해놓는것도 추가요금내야되서 그점이 좀 별로였음 나머진매우만족! ',
  '룸 컨디션은 좋았지만, 추워서 온도를 올리는데 전체적으로 방이 따뜻해지는 느낌은 없고 춥게 잤네요 천장에 

In [ ]:
url = 'https://www.mangoplate.com/restaurants/fvA4SbtNcpuV'
header = {'User-Agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
req = requests.get(url, headers=header)
html2 = req.text

In [15]:
driver.close()